In [1]:
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

In [4]:
data = np.load('Simulations/Epsilon0.1Alpha0.000333333333333Iterations10000000.npy')

data.shape

(10, 10, 2, 2)

In [ ]:
Soft_Stand = data[:, :, 1, 1] 
Hard_Stand = data[:, :, 1, 1] 
Soft_Hit = data[:, :, 1, 1] 
Hard_Hit = data[:, :, 1, 1] 

In [ ]:
def view_state_action_values(data):

    Soft_Stand = data[:, :, 1, 1] 
    Hard_Stand = data[:, :, 1, 1] 
    Soft_Hit = data[:, :, 1, 1] 
    Hard_Hit = data[:, :, 1, 1] 


    # Reorder the matrix for the dealer values (2-10, then Ace)
    # You might need to adjust this depending on how your data is structured
    reordered_matrix = np.vstack([matrix[1:], matrix[0]])

    plt.figure(figsize=(10, 8))
    sns.heatmap(reordered_matrix, annot=True, cmap='viridis')
    plt.title('Your Title Here')
    plt.xlabel('Player Hand Value (12-21)')
    plt.ylabel('Dealer Hand Value (2-10, Ace)')
    plt.show()

    return
